In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/corona-ds/Train_dataset.xlsx
/kaggle/input/corona-ds/Variable_Description.xlsx
/kaggle/input/corona-ds/Test_dataset.xlsx


In [2]:
df = pd.DataFrame(pd.read_excel('/kaggle/input/corona-ds/Train_dataset.xlsx'))
df.head()


,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,...,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
0,1,Bhubaneshwar,Female,Mrs,mansi,YES,1.0,Farmer,Public,2,...,93.0,233.0,82.0,58.0,27,7,3600000.0,1300000,2.0,49.135010
1,2,Bhubaneshwar,Female,Mrs,riya masi,YES,2.0,Farmer,Walk,2,...,56.0,328.0,89.0,68.0,5,6,1600000.0,400000,1.0,51.147880
2,3,Bhubaneshwar,Female,Mrs,sunita,NO,1.0,Cleaner,Public,2,...,137.0,213.0,77.0,43.0,40,6,3400000.0,900000,1.0,73.224000
3,4,Bhubaneshwar,Female,Mrs,anjali @ babli,YES,1.0,Driver,Car,2,...,167.0,275.0,64.0,60.0,27,7,700000.0,2300000,1.0,48.779225
4,5,Bhubaneshwar,Female,Mrs,champa karketta,NO,2.0,Manufacturing,Car,2,...,153.0,331.0,71.0,64.0,32,7,3200000.0,1100000,1.0,87.868800


In [3]:
df = df.set_index("people_ID")
df.head()

,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,Deaths/1M,...,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
people_ID,,,,,,,,,,,,,,,,,,,,,
1,Bhubaneshwar,Female,Mrs,mansi,YES,1.0,Farmer,Public,2,0,...,93.0,233.0,82.0,58.0,27,7,3600000.0,1300000,2.0,49.135010
2,Bhubaneshwar,Female,Mrs,riya masi,YES,2.0,Farmer,Walk,2,0,...,56.0,328.0,89.0,68.0,5,6,1600000.0,400000,1.0,51.147880
3,Bhubaneshwar,Female,Mrs,sunita,NO,1.0,Cleaner,Public,2,0,...,137.0,213.0,77.0,43.0,40,6,3400000.0,900000,1.0,73.224000
4,Bhubaneshwar,Female,Mrs,anjali @ babli,YES,1.0,Driver,Car,2,0,...,167.0,275.0,64.0,60.0,27,7,700000.0,2300000,1.0,48.779225
5,Bhubaneshwar,Female,Mrs,champa karketta,NO,2.0,Manufacturing,Car,2,0,...,153.0,331.0,71.0,64.0,32,7,3200000.0,1100000,1.0,87.868800


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10714 entries, 1 to 22695
Data columns (total 27 columns):
Region                    10714 non-null object
Gender                    10714 non-null object
Designation               10714 non-null object
Name                      10694 non-null object
Married                   10714 non-null object
Children                  10403 non-null float64
Occupation                9967 non-null object
Mode_transport            10711 non-null object
cases/1M                  10714 non-null int64
Deaths/1M                 10714 non-null int64
comorbidity               10488 non-null object
Age                       10714 non-null int64
Coma score                10714 non-null int64
Pulmonary score           10714 non-null object
cardiological pressure    10617 non-null object
Diuresis                  9998 non-null float64
Platelets                 9790 non-null float64
HBB                       10698 non-null float64
d-dimer                   9600

A thing to look at is high cardinality features. 

If we have more than 100 categories for each of these categorical features, it won’t be very useful to use them. It would add dimensions to our dataset and we don’t want to do that.

In [5]:
"""
select_dtypes()
Return a subset of the DataFrame’s columns based on the column dtypes.
"""
max_cardinality = 50
high_cardinality = [col for col in df.select_dtypes(exclude = np.number) if df[col].nunique()> max_cardinality]
high_cardinality

['Name']

In [6]:
df = df.drop(columns = high_cardinality)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10714 entries, 1 to 22695
Data columns (total 26 columns):
Region                    10714 non-null object
Gender                    10714 non-null object
Designation               10714 non-null object
Married                   10714 non-null object
Children                  10403 non-null float64
Occupation                9967 non-null object
Mode_transport            10711 non-null object
cases/1M                  10714 non-null int64
Deaths/1M                 10714 non-null int64
comorbidity               10488 non-null object
Age                       10714 non-null int64
Coma score                10714 non-null int64
Pulmonary score           10714 non-null object
cardiological pressure    10617 non-null object
Diuresis                  9998 non-null float64
Platelets                 9790 non-null float64
HBB                       10698 non-null float64
d-dimer                   9600 non-null float64
Heart rate                9600

In [7]:
df = df.drop(columns = 'salary')
df.columns


Index(['Region', 'Gender', 'Designation', 'Married', 'Children', 'Occupation',
       'Mode_transport', 'cases/1M', 'Deaths/1M', 'comorbidity', 'Age',
       'Coma score', 'Pulmonary score', 'cardiological pressure', 'Diuresis',
       'Platelets', 'HBB', 'd-dimer', 'Heart rate', 'HDL cholesterol',
       'Charlson Index', 'Blood Glucose', 'Insurance', 'FT/month',
       'Infect_Prob'],
      dtype='object')

In [8]:
nan_rows = df[df.isnull().any(1)]
nan_rows

,Region,Gender,Designation,Married,Children,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,...,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,FT/month,Infect_Prob
people_ID,,,,,,,,,,,,,,,,,,,,,
2,Bhubaneshwar,Female,Mrs,YES,2.0,Farmer,Walk,2,0,Diabetes,...,121.0,56.0,328.0,89.0,68.0,5,6,1600000.0,1.0,51.147880
7,Bhubaneshwar,Female,Mrs,YES,2.0,Manufacturing,Car,2,0,Coronary Heart Disease,...,75.0,156.0,311.0,99.0,68.0,23,5,4000000.0,1.0,49.670361
9,Bhubaneshwar,Female,Mrs,YES,2.0,Sales,Public,2,0,Hypertension,...,NaN,156.0,NaN,NaN,37.0,21,7,4500000.0,0.0,48.531922
13,Bhubaneshwar,Female,Mrs,NO,NaN,NaN,Public,2,0,Diabetes,...,39.0,174.0,213.0,98.0,43.0,33,3,NaN,2.0,35.147520
15,Bhubaneshwar,Female,Mrs,YES,2.0,Manufacturing,Public,2,0,Coronary Heart Disease,...,NaN,24.0,NaN,NaN,38.0,4,6,2500000.0,1.0,49.611328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22681,Bengaluru,Male,Mr,YES,2.0,Driver,Public,8,2,Hypertension,...,45.0,114.0,349.0,76.0,52.0,3,4,4800000.0,1.0,59.188997
22687,Bengaluru,Male,Mr,YES,2.0,Manufacturing,Public,8,2,Hypertension,...,29.0,98.0,226.0,89.0,39.0,13,7,3400000.0,1.0,59.374719
22688,Bengaluru,Male,Mr,YES,2.0,Clerk,Public,8,2,Hypertension,...,53.0,77.0,258.0,67.0,63.0,1,4,4900000.0,2.0,58.798970


In [9]:
df = df.dropna()
df.shape

(6791, 25)

In [10]:
import sklearn
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 0.157)

In [11]:
print(df_train.shape)
print(df_test.shape)

(5724, 25)
(1067, 25)


In [12]:
train_x = df_train.iloc[:,0:24]
train_y = df_train.iloc[:,-1]
test_x = df_test.iloc[:, 0:24]
test_y = df_test.iloc[:,-1]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


(5724, 24)
(5724,)
(1067, 24)
(1067,)


In [13]:
type(test_y)

pandas.core.series.Series

In [14]:
train_y = train_y.values.reshape(5724,1)
test_y = test_y.values.reshape(1067,1)

In [15]:
# finding top n features
def findTopFeat(temp_x, temp_y, n): 
    temp_x = train_x
    temp_x = pd.get_dummies(temp_x)
    temp_y = train_y


    label_enc = preprocessing.LabelEncoder()
    temp_y_encoded = label_enc.fit_transform(temp_y)
    bestfeatures = SelectKBest(score_func = chi2, k = 10)
    fit = bestfeatures.fit(temp_x, temp_y_encoded)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(train_x.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print(featureScores.nlargest(n,'Score'))  #print 10 best features

In [16]:
findTopFeat(train_x, train_y,20)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                     Specs         Score
13  cardiological pressure  3.338809e+09
5               Occupation  1.572576e+05
7                 cases/1M  1.211376e+05
6           Mode_transport  9.192257e+04
3                  Married  3.316954e+04
11              Coma score  3.237955e+04
8                Deaths/1M  3.130363e+04
9              comorbidity  1.348932e+04
1                   Gender  1.025203e+04
10                     Age  9.392237e+03
4                 Children  7.099087e+03
2              Designation  6.410822e+03
24                     NaN  4.430000e+03
28                     NaN  4.129778e+03
33                     NaN  4.117883e+03
34                     NaN  4.091910e+03
26                     NaN  4.089326e+03
30                     NaN  4.074045e+03
32                     NaN  4.071361e+03
27                     NaN  4.061680e+03


In [17]:
train_x = train_x[['cardiological pressure','cases/1M', 'Coma score', 
                   'Platelets', 'comorbidity','Gender', 'Age','Heart rate', 'Pulmonary score']]
test_x = test_x[['cardiological pressure', 'cases/1M', 'Coma score', 
                   'Platelets', 'comorbidity','Gender', 'Age','Heart rate', 'Pulmonary score']]
print(train_x.shape)
print(test_x.shape)

(5724, 9)
(1067, 9)


In [18]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5724 entries, 3858 to 14375
Data columns (total 9 columns):
cardiological pressure    5724 non-null object
cases/1M                  5724 non-null int64
Coma score                5724 non-null int64
Platelets                 5724 non-null float64
comorbidity               5724 non-null object
Gender                    5724 non-null object
Age                       5724 non-null int64
Heart rate                5724 non-null float64
Pulmonary score           5724 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 447.2+ KB


In [19]:
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
train_x.shape

(5724, 19)

In [20]:
findTopFeat(train_x, train_y, 20)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                                 Specs         Score
2                            Platelets  91922.567042
3                                  Age  33169.544406
4                           Heart rate  13489.323012
0                             cases/1M  10252.030934
1                           Coma score   7099.087132
15                Pulmonary score_<100   3502.582324
8      cardiological pressure_Stage-02   3502.582324
10                comorbidity_Diabetes   3478.084812
11            comorbidity_Hypertension   3464.375529
7      cardiological pressure_Stage-01   3439.554839
16                Pulmonary score_<200   3439.554839
5      cardiological pressure_Elevated   3429.051414
17                Pulmonary score_<300   3429.051414
12                    comorbidity_None   3417.918872
18                Pulmonary score_<400   3398.285332
6        cardiological pressure_Normal   3398.285332
9   comorbidity_Coronary Heart Disease   3396.728870
14                         Gender_Male   2337.

In [21]:
features = [ 'Coma score','Platelets','Age','Heart rate',
            'comorbidity_Hypertension','comorbidity_Diabetes']

In [22]:
train_x = train_x[features]
test_x = test_x[features]

In [23]:
train_x.shape

(5724, 6)

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train_x[features] = scaler.fit_transform(train_x[features])
test_x[features] = scaler.fit_transform(test_x[features])
#train_y = scaler.fit_transform(train_y)
#test_y = scaler.fit_transform(test_y)
print(train_y.shape)
print(test_y.shape)

(5724, 1)
(1067, 1)


In [25]:
train_x.shape

(5724, 6)

In [26]:
test_x.shape

(1067, 6)

In [27]:
test_x

,Coma score,Platelets,Age,Heart rate,comorbidity_Hypertension,comorbidity_Diabetes
people_ID,,,,,,
477,0.083333,0.848276,0.615385,0.64,0.0,0.0
652,0.666667,0.903448,0.000000,0.70,0.0,1.0
15640,0.583333,0.641379,0.000000,0.22,0.0,0.0
2667,0.333333,0.468966,0.307692,0.88,0.0,1.0
9063,0.333333,0.627586,0.596154,0.82,0.0,0.0
...,...,...,...,...,...,...
13204,1.000000,0.048276,0.961538,0.50,1.0,0.0
2011,0.750000,0.834483,0.615385,0.06,0.0,1.0
2869,0.750000,0.400000,0.634615,0.60,0.0,0.0


In [28]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(train_x, train_y)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [29]:
y_predicted = regr.predict(test_x)

In [30]:
#y_predicted = scaler.inverse_transform(y_predicted)

In [31]:
test_y.shape

(1067, 1)

In [32]:
y_predicted = y_predicted.reshape(y_predicted.shape[0],1)
y_predicted.shape

(1067, 1)

In [33]:
from sklearn.metrics import r2_score
r2_score(test_y, y_predicted)  

0.33202235247615564

In [34]:
test = pd.DataFrame(pd.read_excel('/kaggle/input/corona-ds/Test_dataset.xlsx'))

In [35]:
test.shape

(14498, 27)

In [36]:
test.fillna(test.median())

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,...,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month
0,5942,Delhi,Female,Mrs,smt rekha prajapat,YES,2,Driver,Public,4,...,153,196,240,85,53,17,3,3900000,1300000,1
1,18664,Delhi,Male,Mr,nirmal,YES,2,Legal,Walk,4,...,95,138,241,81,61,2,5,1800000,1300000,1
2,5603,Delhi,Female,Mrs,pinky,YES,2,Sales,Car,4,...,40,166,236,88,47,24,3,5000000,2000000,2
3,5649,Delhi,Female,Mrs,pooja @aafrin,YES,2,Sales,Car,4,...,78,83,211,87,52,13,6,3100000,600000,2
4,5099,Delhi,Female,Mrs,anjali,YES,2,Business,Car,4,...,109,207,312,94,68,39,5,2300000,1500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14493,23737,Thiruvananthapuram,Male,Mr,hanuman sing,YES,0,Cleaner,Walk,11,...,16,36,346,66,45,34,3,4200000,800000,2
14494,11171,Thiruvananthapuram,Female,Mrs,gurupreet kaur @ nainsi,YES,0,Clerk,Car,11,...,22,29,278,94,37,23,6,4500000,1800000,2
14495,23243,Thiruvananthapuram,Male,Mr,sonu pandey,YES,0,Business,Public,11,...,15,28,294,73,56,6,6,4300000,800000,1
14496,11199,Thiruvananthapuram,Female,Mrs,nikky,YES,0,Driver,Car,11,...,31,24,227,83,60,32,7,2400000,1800000,2


In [37]:
test = test.set_index("people_ID")
test.head()

,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,Deaths/1M,...,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month
people_ID,,,,,,,,,,,,,,,,,,,,,
5942,Delhi,Female,Mrs,smt rekha prajapat,YES,2,Driver,Public,4,1,...,153,196,240,85,53,17,3,3900000,1300000,1
18664,Delhi,Male,Mr,nirmal,YES,2,Legal,Walk,4,1,...,95,138,241,81,61,2,5,1800000,1300000,1
5603,Delhi,Female,Mrs,pinky,YES,2,Sales,Car,4,1,...,40,166,236,88,47,24,3,5000000,2000000,2
5649,Delhi,Female,Mrs,pooja @aafrin,YES,2,Sales,Car,4,1,...,78,83,211,87,52,13,6,3100000,600000,2
5099,Delhi,Female,Mrs,anjali,YES,2,Business,Car,4,1,...,109,207,312,94,68,39,5,2300000,1500000,1


In [38]:
test = test.drop(columns = 'Name')


In [39]:
test = test.drop(columns = 'salary')

In [40]:
test = test[['cardiological pressure','cases/1M', 'Coma score', 
                   'Platelets', 'comorbidity','Gender', 'Age','Heart rate', 'Pulmonary score']]

In [41]:
test = pd.get_dummies(test)


In [42]:
test.fillna(test.median())

,cases/1M,Coma score,Platelets,Age,Heart rate,cardiological pressure_Elevated,cardiological pressure_Normal,cardiological pressure_Stage-01,cardiological pressure_Stage-02,comorbidity_Coronary Heart Disease,comorbidity_Diabetes,comorbidity_Hypertension,comorbidity_None,Gender_Female,Gender_Male,Pulmonary score_<100,Pulmonary score_<200,Pulmonary score_<300,Pulmonary score_<400
people_ID,,,,,,,,,,,,,,,,,,,
5942,4,3,153,52,85,0,0,1,0,0,1,0,0,1,0,0,1,0,0
18664,4,11,95,53,81,0,0,1,0,0,1,0,0,0,1,0,1,0,0
5603,4,9,40,35,88,0,0,0,1,0,1,0,0,1,0,1,0,0,0
5649,4,12,78,31,87,1,0,0,0,0,0,0,1,1,0,0,0,1,0
5099,4,6,109,51,94,0,0,1,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23737,11,5,16,55,66,0,1,0,0,1,0,0,0,0,1,0,0,0,1
11171,11,4,22,44,94,1,0,0,0,0,0,0,1,1,0,0,0,1,0
23243,11,6,15,46,73,1,0,0,0,0,0,0,1,0,1,0,0,1,0


In [43]:
test.shape

(14498, 19)

In [44]:
test = test[features]
test.shape

(14498, 6)

In [45]:
test[features] = scaler.fit_transform(test[features])

In [46]:
test

,Coma score,Platelets,Age,Heart rate,comorbidity_Hypertension,comorbidity_Diabetes
people_ID,,,,,,
5942,0.000000,0.951724,0.692308,0.70,0.0,1.0
18664,0.666667,0.551724,0.711538,0.62,0.0,1.0
5603,0.500000,0.172414,0.365385,0.76,0.0,1.0
5649,0.750000,0.434483,0.288462,0.74,0.0,0.0
5099,0.250000,0.648276,0.673077,0.88,0.0,1.0
...,...,...,...,...,...,...
23737,0.166667,0.006897,0.750000,0.32,0.0,0.0
11171,0.083333,0.048276,0.538462,0.88,0.0,0.0
23243,0.250000,0.000000,0.576923,0.46,0.0,0.0


In [47]:
test.shape

(14498, 6)

In [48]:
y_out = regr.predict(test)

In [49]:
y_out

array([48.82575423, 48.82575423, 48.96098192, ..., 48.82575423,
       49.59619701, 51.29639699])

In [50]:
y_out = y_out.reshape(y_out.shape[0],1)
y_out.shape

(14498, 1)

In [51]:
y_out = pd.DataFrame({'infect_prob': y_out[:, 0]})

In [52]:
y_out

,infect_prob
0,48.825754
1,48.825754
2,48.960982
3,51.296397
4,48.825754
...,...
14493,48.825754
14494,48.825754
14495,48.825754
14496,49.596197


In [53]:
people = pd.DataFrame(pd.read_excel('/kaggle/input/corona-ds/Test_dataset.xlsx'))
people = people['people_ID']
people = people.values.reshape(people.shape[0],1)
people.shape

(14498, 1)

In [54]:
people = pd.DataFrame({'people_ID': people[:, 0]})

In [55]:
people

,people_ID
0,5942
1,18664
2,5603
3,5649
4,5099
...,...
14493,23737
14494,11171
14495,23243
14496,11199


In [56]:
frames = [people, y_out]
result = pd.concat([people, y_out], axis=1, sort=False)
result

,people_ID,infect_prob
0,5942,48.825754
1,18664,48.825754
2,5603,48.960982
3,5649,51.296397
4,5099,48.825754
...,...,...
14493,23737,48.825754
14494,11171,48.825754
14495,23243,48.825754
14496,11199,49.596197


In [57]:
result.to_csv("result_part_one.csv", index=False)